In [1]:
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPooling2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
train_data_dir = 'D:/Tea/面部表情识别/Images/train'
validation_data_dir = 'D:/Tea/面部表情识别/Images/validation'
img_width, img_height = 32, 32

In [ ]:
#数据预处理
# 图像预处理
batch_size = 16
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [ ]:
#搭建模型
model = Sequential()

############################################################
model.add(Conv2D(filters=64, 
                kernel_size = (3, 3), 
                activation='relu', 
                input_shape=(img_width, img_height, 3)))
model.add(Conv2D(filters=64, 
                 kernel_size = (3, 3),
                 activation='relu',
                 padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

############################################################
model.add(Conv2D(filters=128, 
                 kernel_size = (3, 3),
                 activation='relu',
                 padding = 'same'))
model.add(Conv2D(filters=128, 
                 kernel_size = (3, 3),
                 activation='relu',
                 padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

############################################################
model.add(Conv2D(filters=256,
                 kernel_size = (3, 3),
                 activation='relu',
                 padding = 'same'))
model.add(Conv2D(filters=256,
                 kernel_size = (3, 3),
                 activation='relu',
                 padding = 'same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

#全连接层
#展平层
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(train_generator.num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 64)        36928     
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 15, 15, 128)       7

In [ ]:
#编译模型
model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#训练模型
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator)

Epoch 1/100
1795/1795 [==============================] - 255s 141ms/step - loss: 1.8130 - accuracy: 0.2523 - val_loss: 1.7289 - val_accuracy: 0.2653
Epoch 2/100
1795/1795 [==============================] - 251s 140ms/step - loss: 1.6923 - accuracy: 0.3037 - val_loss: 1.6758 - val_accuracy: 0.3357
Epoch 3/100
1795/1795 [==============================] - 250s 139ms/step - loss: 1.6308 - accuracy: 0.3383 - val_loss: 1.6210 - val_accuracy: 0.3527
Epoch 4/100
1795/1795 [==============================] - 252s 140ms/step - loss: 1.5482 - accuracy: 0.3772 - val_loss: 1.7208 - val_accuracy: 0.2644
Epoch 5/100
1795/1795 [==============================] - 253s 141ms/step - loss: 1.4564 - accuracy: 0.4062 - val_loss: 1.6908 - val_accuracy: 0.3396
Epoch 6/100
 898/1795 [==============>...............] - ETA: 2:04 - loss: 1.4065 - accuracy: 0.4366

In [ ]:
score = model.evaluate_generator(validation_generator, steps=validation_generator.n // batch_size)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()